In [4]:
from typing import List
from functools import reduce
import numpy as np


def cumulative_gain(relevance: List[float], k: int) -> float:
    """Score is cumulative gain at k (CG@k)

    Parameters
    ----------
    relevance:  `List[float]`
        Relevance labels (Ranks)
    k : `int`
        Number of elements to be counted

    Returns
    -------
    score : float
    """
    score = reduce(lambda x, y: x+y, relevance[:k])
    return score


In [3]:
sorted([2,3,4], reverse=True)

[4, 3, 2]

In [7]:
relevance = [0.99, 0.94, 0.88, 0.74, 0.71, 0.68]
cumulative_gain(relevance, 5)

4.26

In [97]:
from typing import List
import numpy as np
from functools import reduce


def discounted_cumulative_gain(relevance: List[float], k: int, method: str = "standard") -> float:
    """Discounted Cumulative Gain

    Parameters
    ----------
    relevance : `List[float]`
        Video relevance list
    k : `int`
        Count relevance to compute
    method : `str`, optional
        Metric implementation method, takes the values​​
        `standard` - adds weight to the denominator
        `industry` - adds weights to the numerator and denominator
        `raise ValueError` - for any value

    Returns
    -------
    score : `float`
        Metric score
    """
    relevance = relevance[:k]
    
    if method == 'standard':
        dcg_standard_acc = lambda acc, idx: acc + relevance[idx] / np.log2(idx + 2)
        score = reduce(dcg_standard_acc, range(len(relevance)), 0)

    elif method == 'industry':
        dcg_industry_acc = lambda acc, idx: acc + (2**relevance[idx] - 1) / np.log2(idx + 2)
        score = reduce(dcg_industry_acc, range(len(relevance)), 0)
        
    else:
        raise ValueError
    
    return score

In [ ]:
from typing import List
import numpy as np
from functools import reduce

def discounted_cumulative_gain(relevance: List[float], k: int, method: str = "standard") -> float:
    """Discounted Cumulative Gain

    Parameters
    ----------
    relevance : List[float]
        Video relevance list
    k : int
        Count relevance to compute
    method : str, optional
        Metric implementation method, takes the values:
        'standard' - adds weight to the denominator
        'industry' - adds weights to the numerator and denominator
        'raise ValueError' - for any value

    Returns
    -------
    score : float
        Metric score
    """
    relevance = relevance[:k]

    if method == 'standard':
        def dcg_standard_acc(acc, idx):
            return acc + relevance[idx] / np.log2(idx + 2)
        score = reduce(dcg_standard_acc, range(len(relevance)), 0)

    elif method == 'industry':
        def dcg_industry_acc(acc, idx):
            return acc + (2**relevance[idx] - 1) / np.log2(idx + 2)
        score = reduce(dcg_industry_acc, range(len(relevance)), 0)

    else:
        raise ValueError("Invalid method")

    return score


In [98]:
relevance = [0.99, 0.94, 0.88, 0.74, 0.71, 0.68]
k = 5
method = 'standard'
print(discounted_cumulative_gain(relevance, k, method))



2.6164401144680056


In [36]:
from typing import List
from functools import reduce
import numpy as np

def normalized_dcg(relevance: List[float], k: int, method: str = "standard") -> float:
    """Discounted Cumulative Gain

    Parameters
    ----------
    relevance : List[float]
        Video relevance list
    k : int
        Count relevance to compute
    method : str, optional
        Metric implementation method, takes the values:
        'standard' - adds weight to the denominator
        'industry' - adds weights to the numerator and denominator
        'raise ValueError' - for any value

    Returns
    -------
    score : float
        Metric score
    """
    relevance_sorted = sorted(relevance, reverse=True)[:k]
    relevance = relevance[:k]
    
    if method == 'standard':
        def dcg_standard_acc(acc, idx):
            return acc + relevance[idx] / np.log2(idx + 2)
        dcg = reduce(dcg_standard_acc, range(len(relevance)), 0)
        relevance = relevance_sorted
        idcg = reduce(dcg_standard_acc, range(len(relevance)), 0)
        score = dcg / idcg

    elif method == 'industry':
        def dcg_industry_acc(acc, idx):
            return acc + (2**relevance[idx] - 1) / np.log2(idx + 2)
        dcg = reduce(dcg_industry_acc, range(len(relevance)), 0)
        relevance = relevance_sorted
        idcg = reduce(dcg_industry_acc, range(len(relevance)), 0)
    else:
        raise ValueError("Invalid method")

    return score


In [37]:
>>> relevance = [0.99, 0.94, 0.74, 0.88, 0.71, 0.68]
>>> k = 5
>>> method = 'standard'
>>> print(normalized_dcg(relevance, k, method))

# 0.9962...

0.9962906539247512


In [43]:
from typing import List
from functools import reduce
import numpy as np

def avg_ndcg(list_relevances: List[List[float]], k: int, method: str = 'standard') -> float:
    """Average nDCG

    Parameters
    ----------
    list_relevances : `List[List[float]]`
        Video relevance matrix for various queries
    k : `int`
        Count relevance to compute
    method : `str`, optional
        Metric implementation method, takes the values ​​\
        `standard` - adds weight to the denominator\
        `industry` - adds weights to the numerator and denominator\
        `raise ValueError` - for any value

    Returns
    -------
    score : `float`
        Metric score
    """
    cum_score = 0
    n = len(list_relevances)

    for relevance in list_relevances:
        
        relevance_sorted = sorted(relevance, reverse=True)[:k]
        relevance = relevance[:k]

        if method == 'standard':
            def dcg_standard_acc(acc, idx):
                return acc + relevance[idx] / np.log2(idx + 2)
                
            dcg = reduce(dcg_standard_acc, range(len(relevance)), 0)
            relevance = relevance_sorted
            idcg = reduce(dcg_standard_acc, range(len(relevance)), 0)
            
        elif method == 'industry':
            def dcg_industry_acc(acc, idx):
                return acc + (2**relevance[idx] - 1) / np.log2(idx + 2)
                
            dcg = reduce(dcg_industry_acc, range(len(relevance)), 0)
            relevance = relevance_sorted
            idcg = reduce(dcg_industry_acc, range(len(relevance)), 0)
            
        else:
            raise ValueError("Invalid method")
        score = dcg / idcg
        cum_score += score
    result = cum_score / n
    
    return result



In [44]:
>>> list_relevances = [
        [0.99, 0.94, 0.88, 0.89, 0.72, 0.65],
        [0.99, 0.92, 0.93, 0.74, 0.61, 0.68], 
        [0.99, 0.96, 0.81, 0.73, 0.76, 0.69]
    ]  
>>> k = 5
>>> method = 'standard'
>>> print(avg_ndcg(list_relevances, k, method))

# 0.99613...

0.9961322104432755
